In [ ]:
import os.path as osp

import torch
import torch.nn.functional as F
from torch_geometric.datasets import Amazon
import torch_geometric.transforms as T
from torch_geometric.nn import GATConv
import numpy as np

dataset = "Computers"
path = osp.join('data', dataset)
dataset = Amazon(path, dataset, transform=T.NormalizeFeatures())
dataset.shuffle()
data = dataset[0]

data = dataset[0]
TRAIN_END = 20 * dataset.num_classes
VAL_END = TRAIN_END + 500
TEST_END = VAL_END + 1000
data = dataset[0]
data.train_mask = torch.zeros([data.num_nodes,], dtype=torch.bool)
data.train_mask[:TRAIN_END] = True
data.val_mask = torch.zeros([data.num_nodes,], dtype=torch.bool)
data.val_mask[TRAIN_END:VAL_END] = True
data.test_mask = torch.zeros([data.num_nodes,], dtype=torch.bool)
data.test_mask[VAL_END:TEST_END] = True


class Net(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(Net, self).__init__()

        self.conv1 = GATConv(in_channels, 8, heads=8, dropout=0.6)
        # On the Pubmed dataset, use heads=8 in conv2.
        self.conv2 = GATConv(8 * 8, out_channels, heads=1, concat=False,
                             dropout=0.6)

    def forward(self, x, edge_index):
        x = F.dropout(x, p=0.6, training=self.training)
        x = F.elu(self.conv1(x, edge_index))
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv2(x, edge_index)
        return F.log_softmax(x, dim=-1)


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net(dataset.num_features, dataset.num_classes).to(device)
data = data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)


def train(data):
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()


@torch.no_grad()
def test(data):
    model.eval()
    out, accs = model(data.x, data.edge_index), []
    for _, mask in data('train_mask', 'val_mask', 'test_mask'):
        acc = float((out[mask].argmax(-1) == data.y[mask]).sum() / mask.sum())
        accs.append(acc)
    return accs


print()
avg = np.empty([1,])
max_val = 0
early_stop_cnt = 0
for epoch in range(1, 1001):
    train(data)
    train_acc, val_acc, test_acc = test(data)

    if val_acc > max_val:
        early_stop_cnt = 0
        max_val = val_acc
    else:
        early_stop_cnt += 1
        if (early_stop_cnt == 100):
            print(f'Early Stopping at epoch {epoch:03d}, Val: {val_acc:.4f}, Test: {test_acc:.4f}')
            break

    avg = np.append(avg, test_acc)
    last_hun = np.average(avg[len(avg)-100:])

    print(f'Epoch: {epoch:03d}, Train: {train_acc:.4f}, Val: {val_acc:.4f}, '
          f'Test: {test_acc:.4f}, Avg100: {last_hun:.4f}')


Epoch: 001, Train: 0.3600, Val: 0.3900, Test: 0.3860, Avg100: 0.1930
Epoch: 002, Train: 0.3600, Val: 0.3900, Test: 0.3860, Avg100: 0.2573
Epoch: 003, Train: 0.3600, Val: 0.3900, Test: 0.3860, Avg100: 0.2895
Epoch: 004, Train: 0.3600, Val: 0.3900, Test: 0.3860, Avg100: 0.3088
Epoch: 005, Train: 0.3600, Val: 0.3900, Test: 0.3860, Avg100: 0.3217
Epoch: 006, Train: 0.3600, Val: 0.3900, Test: 0.3860, Avg100: 0.3309
Epoch: 007, Train: 0.3600, Val: 0.3900, Test: 0.3860, Avg100: 0.3378
Epoch: 008, Train: 0.3600, Val: 0.3900, Test: 0.3860, Avg100: 0.3431
Epoch: 009, Train: 0.3600, Val: 0.3900, Test: 0.3860, Avg100: 0.3474
Epoch: 010, Train: 0.3600, Val: 0.3900, Test: 0.3860, Avg100: 0.3509
Epoch: 011, Train: 0.3600, Val: 0.3900, Test: 0.3860, Avg100: 0.3538
Epoch: 012, Train: 0.3600, Val: 0.3900, Test: 0.3860, Avg100: 0.3563
Epoch: 013, Train: 0.3600, Val: 0.3900, Test: 0.3860, Avg100: 0.3584
Epoch: 014, Train: 0.3600, Val: 0.3900, Test: 0.3860, Avg100: 0.3603
Epoch: 015, Train: 0.3600, Val: 0

Epoch: 121, Train: 0.4650, Val: 0.5080, Test: 0.4910, Avg100: 0.4032
Epoch: 122, Train: 0.5000, Val: 0.5300, Test: 0.5170, Avg100: 0.4045
Epoch: 123, Train: 0.5200, Val: 0.5520, Test: 0.5360, Avg100: 0.4060
Epoch: 124, Train: 0.5300, Val: 0.5720, Test: 0.5430, Avg100: 0.4075
Epoch: 125, Train: 0.5350, Val: 0.5720, Test: 0.5440, Avg100: 0.4091
Epoch: 126, Train: 0.5450, Val: 0.5740, Test: 0.5460, Avg100: 0.4107
Epoch: 127, Train: 0.5300, Val: 0.5400, Test: 0.5260, Avg100: 0.4121
Epoch: 128, Train: 0.5050, Val: 0.5000, Test: 0.4940, Avg100: 0.4132
Epoch: 129, Train: 0.5000, Val: 0.4960, Test: 0.4910, Avg100: 0.4143
Epoch: 130, Train: 0.5050, Val: 0.4860, Test: 0.4910, Avg100: 0.4153
Epoch: 131, Train: 0.4950, Val: 0.4900, Test: 0.4930, Avg100: 0.4164
Epoch: 132, Train: 0.5100, Val: 0.4960, Test: 0.4910, Avg100: 0.4174
Epoch: 133, Train: 0.5050, Val: 0.5040, Test: 0.4900, Avg100: 0.4185
Epoch: 134, Train: 0.4950, Val: 0.4940, Test: 0.4930, Avg100: 0.4195
Epoch: 135, Train: 0.5050, Val: 0.